# Siamese Network
------------------------------

Constrastive loss: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf

Siamese Network for one shot learning: https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf

### Environment

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

### Dataset

In [146]:
sv_embeds = np.load("xvector_embeds/sv_embeds.npy")

In [35]:
import torch.utils.data as data

class EmbedDataset(data.Dataset):
    def __init__(self, embeds, labels):
        super().__init__()
        self.embeds = embeds
        self.labels = labels
        
    def __getitem__(self, index):
        return self.embeds[index], self.labels[index]
    
    def __len__(self):
        return self.embeds.shape[0]

In [298]:
import itertools
def gen_pairs(data, enr_idx):
    labels = data.label
    label2idx= data.groupby('label').apply(lambda x: x.idx.tolist()).to_dict()
    # 1: enrolled, 0: imposter
    label2idx[1] += enr_idx
    positive_pairs = np.array(list(itertools.combinations(label2idx[1], 2)))
    negative_pairs = np.array(list(itertools.product(label2idx[0], label2idx[1])))
    
    return positive_pairs, negative_pairs

In [299]:
enr_idx = np.load("enroll_idx.npy").tolist()
adapt_data = pd.read_csv("adapt_data.csv")
test_data = pd.read_csv("test_data.csv")
ood_data = pd.read_csv("ood_data.csv")

In [300]:
positive_pairs, negative_pairs = gen_pairs(adapt_data, enr_idx)

### Model Define

In [301]:
import torch.nn as nn
import torch.nn.functional as F

class SiameseNet(nn.Module):
    def __init__(self, in_dims, n_layers):
        super().__init__()
        
        self.input_layer = nn.Sequential(
            nn.Linear(in_dims, 1*in_dims),
            nn.PReLU()
        )
        
        hidden_layer = [nn.Linear(1*in_dims, 1*in_dims),
            nn.BatchNorm1d(1*in_dims),
            nn.PReLU()] * n_layers
        
        self.hidden_layer = nn.Sequential(*hidden_layer)
    
        self.embedding_layer = nn.Sequential(
            nn.Linear(1*in_dims, 1),
        )
        
    def embed(self, x):
        x = self.input_layer(x)
        x = self.hidden_layer(x)
        
        return x
    
    def score(self, embed1, embed2):
        dist = embed1.sub(embed2).abs()
        dist = torch.clamp(dist, min=1e-16) # for numerical stability
        weighted_dist = self.embedding_layer(dist)
        
        return weighted_dist.squeeze(1)
        
    def forward(self, x1, x2):           
        embed1 = self.embed(x1)
        embed2 = self.embed(x2)
        p = torch.sigmoid(self.score(embed1, embed2))
        
        return p
        
    def batch_score(self, x1, x2):
        embed1 = self.embed(x1)
        embed2 = self.embed(x2)

        dist = embed1.unsqueeze(1).sub(embed2.unsqueeze(0)).abs()
        dist = dist.mean(dim=0)
        dist = torch.clamp(dist, min=1e-16) # for numerical stability
        weighted_dist = self.embedding_layer(dist)
        p = torch.sigmoid(weighted_dist.squeeze(1))
        
        return p

### Train funcs

In [302]:
def constrastive_loss(n1, n2, label, margin):
    dist_square = (n1 - n2).pow(2).sum(1)
    dist_square = torch.clamp(dist_square, min=1e-16)
    dist = dist_square.sqrt()

    loss = torch.mean(
        (1.0-label)*dist_square + (label)*torch.pow(torch.clamp(margin-dist, min=0.0), 2)
    )
    return loss

In [303]:
model = SiameseNet(512, 3)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
criterion = nn.BCELoss()

In [304]:
import random
import torch

model.train()
batch_size = 32
n_epochs = 100
min_n_pairs = min(len(positive_pairs), len(negative_pairs))
for epoch_i in range(n_epochs):
    loss_sum = 0
    n_corrects = 0
    total = 0
    np.random.shuffle(positive_pairs)
    np.random.shuffle(negative_pairs)
    for batch_i in range(min_n_pairs//batch_size):
        p_pairs = random.choices(positive_pairs, k=batch_size//2)
        n_pairs = random.choices(negative_pairs, k=batch_size//2)
        x1_idx, x2_idx = np.concatenate([p_pairs, n_pairs], axis=0).T
        x1 = torch.from_numpy(sv_embeds[x1_idx]).float()
        x2 = torch.from_numpy(sv_embeds[x2_idx]).float()
        y = torch.from_numpy(np.concatenate(
            [np.ones(len(p_pairs)), np.zeros(len(n_pairs))], 
            axis=0)).float()
        if torch.cuda.is_available():
            model = model.cuda()
            x1 = x1.cuda()
            x2 = x2.cuda()
            y = y.cuda()

        optimizer.zero_grad()
        score = model(x1, x2)
        loss = criterion(score, y)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        pred = score > 0.5
        n_corrects += pred.eq(y.byte()).sum().item()
        total += y.size(0)
    print("[epoch {}] loss: {}, acc: {}".format(epoch_i, loss_sum, n_corrects/total))

[epoch 0] loss: 18.17130588076543, acc: 0.9891197643979057
[epoch 1] loss: 0.7616312809986994, acc: 1.0
[epoch 2] loss: 0.4748801965615712, acc: 1.0
[epoch 3] loss: 0.38172781449975446, acc: 1.0
[epoch 4] loss: 0.31972661372856237, acc: 1.0


KeyboardInterrupt: 

enrollment uttrs를 고려해서 실험해야한다.!

In [312]:
enr_embeds = sv_embeds[enr_idx]

In [338]:
test_embeds = sv_embeds[test_data.idx]
test_label = np.array(test_data.label)

In [340]:
test_embeds = sv_embeds[ood_data.idx]
test_label = np.array(ood_data.label)

In [1]:
model.eval()
n_corrects = 0
x1 = torch.from_numpy(enr_embeds)
x2 = torch.from_numpy(test_embeds)
y = torch.from_numpy((test_label))
if torch.cuda.is_available():
    model = model.cuda()
    x1 = x1.cuda()
    x2 = x2.cuda()
    y = y.cuda()

# 현재 구현은 sigmoid 값을 score로 삼는데 실제로는 embedding 뽑아서 scosre를 계산해야 한다.
score = model.batch_score(x1, x2)
pred = score > 0.5
n_corrects = pred.eq(y.byte()).sum().item()
acc = n_corrects / len(y)
print(acc)

NameError: name 'model' is not defined

In [321]:
score

tensor([0.3728, 0.3401, 0.3068, 0.2819, 0.3569, 0.3860, 0.3967, 0.2959, 0.3875,
        0.4198, 0.4520, 0.3975, 0.3651, 0.2991, 0.3065, 0.5325, 0.5283, 0.3490,
        0.3703, 0.5249, 0.5270, 0.4470, 0.5232, 0.3937, 0.4858, 0.4193, 0.3550,
        0.3313, 0.4034, 0.3678, 0.3551, 0.4013, 0.4269, 0.3869, 0.2983, 0.4207,
        0.4489, 0.3804, 0.3793, 0.3216, 0.3897, 0.3475, 0.5300, 0.3641, 0.3853,
        0.3620, 0.2916, 0.3063, 0.4578, 0.4046, 0.3386, 0.3514, 0.2563, 0.3537,
        0.3338, 0.3369, 0.3602, 0.5782, 0.3968, 0.4460, 0.3332, 0.3262, 0.3202,
        0.3816, 0.5368, 0.3954, 0.3534, 0.3281, 0.3347, 0.5427, 0.3015, 0.3931,
        0.3744, 0.2921, 0.3277, 0.5374, 0.4065, 0.3495, 0.3786, 0.5245, 0.4406,
        0.5372, 0.5214, 0.3841, 0.5277, 0.5282, 0.3963, 0.4472, 0.2991, 0.4222,
        0.5574, 0.5415, 0.3005, 0.4028, 0.3845, 0.4371, 0.3874, 0.5367, 0.5259,
        0.5311, 0.3478, 0.3845, 0.3594, 0.4211, 0.3881, 0.3907, 0.3717, 0.3910,
        0.3703, 0.4068, 0.5335, 0.5789, 